In [ ]:
import os.path
import base64
import json
import re
import time
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import logging
import requests
import datetime
import os
import subprocess
import sys

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly','https://www.googleapis.com/auth/gmail.modify']

def readDailyReport(sender="",date="",onlyUnread=False):
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    criteria = ""
    if onlyUnread: criteria="is:unread"
    
    new_productions = []

    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    try:
        # Call the Gmail API
        service = build('gmail', 'v1', credentials=creds)
        results = service.users().messages().list(userId='me', labelIds=['INBOX'], q=criteria).execute()
        messages = results.get('messages',[]);
        if not messages:
            print('No messages.')
        else:
            message_count = 0
            for message in messages:
                msg = service.users().messages().get(userId='me', id=message['id']).execute()                
                email_data = msg['payload']['headers']
                email_sender = [values["value"] for values in email_data if values["name"] == "From"][0]
                if sender and (sender in email_sender):
                    email_date = [values["value"] for values in email_data if values["name"] == "Date"][0]
                    email_subject = [values["value"] for values in email_data if values["name"] == "Subject"][0]
                    if (not date) or (date in email_subject):
                        print(f"Read following email (ID:{message['id']}):")
                        print("Sender:",sender, "Date:", email_date, "Subject:", email_subject)
                        for i,part in enumerate(msg['payload']['parts']):
                            try:
                                data = part['body']["data"]
                                byte_code = base64.urlsafe_b64decode(data)

                                text = byte_code.decode("utf-8").split("MonteCarlo")[0]
                                if i == 0: 
                                    paths = str(text).split("path:")[1:]
                                    for path in paths:
                                        line = path.split("\n")[0].split("QC")[0]+"QC/"
                                        new_productions.append(line)

                                # mark the message as read (optional)
                                #msg  = service.users().messages().modify(userId='me', id=message['id'], body={'removeLabelIds': ['UNREAD']}).execute()                                                       
                            except BaseException as error:
                                pass
    except Exception as error:
        print(f'An error occurred: {error}')
        
    if len(new_productions):
        print("Found following new productions:")
        for prod in new_productions:
            print(prod)
    else:
        print("No new productions found!")
    
    return new_productions

In [ ]:
today = datetime.date.today().strftime("%d.%m.%Y")
new_productions = readDailyReport("berkin.ulukutlu@cern.ch",today,onlyUnread=False)
years = list(set([path.split("/")[3] for path in new_productions]))
passes = [list(set([path.split("/")[6] for path in new_productions if path.split("/")[3] == year])) for year in years]
periods = [list(set([path.split("/")[4] for path in new_productions if path.split("/")[3] == year])) for year in years]
print(years, periods, passes)

In [ ]:
# Downloading
local_dir = "/cave/alice-tpc-qc/data/"
for path in new_productions:
    year = path.split("/")[3]
    period = path.split("/")[4]
    apass = path.split("/")[6]
    runNumber = path.split("/")[5]
    target_path = path + "001/QC_fullrun.root"
    local_path  = local_dir+"/"+year+"/"+period+"/"+apass+"/"+runNumber+".root"
    print("Downloading " + target_path)
    print("Executing:",["alien.py", "cp", "alien:" + target_path, "file:"+local_path])
    #subprocess.run(["alien.py", "cp", "alien:" + target_path, "file:"+local_path])
    time.sleep(0.1)

# Plotting


In [ ]:
path = "/cave/alice-tpc-qc/data//2023/LHC23zs/apass1/539647.root".split("/")
"/"+os.path.join(*path[:-1])+"/"


In [ ]:
entry = list(set(["/"+os.path.join(*path[:-1])+"/"]))[0]
entry.split("/")[-3]

In [ ]:
from IPython.display import display, Markdown, HTML
import datetime

                                                            
def createMessage():
    import os
    def generateLink(path):
        link = "https://alice-tpc-qc.web.cern.ch/reports/"+path[path.find('202'):]
        return link
    
    dirs = []
    files = []
    directory = "/cave/alice-tpc-qc/data/2023/" #LOCALDIR
    for root, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith('.html'):
                fname = os.path.join(root, filename)
                dirs.append(root)
                files.append(fname)
    dirs = list(set(sorted(dirs)))
    files = sorted(files)
    myText = ["## Daily TPC Async QC Report\n"]
    for di in dirs:
        myTitle = "### ["+di[di.find("LHC"):]+"]("+generateLink(di)+")\n"
        myText.append(myTitle)
        myList = ""
        for file in files:
            if di in file and "/" not in file[len(di)+1:]:
                myList += ("["+file.split("/")[-1]+"]("+generateLink(file)+") - ")
        myText.append(myList[:-3])
    myText = '\n'.join(map(str,myText))
    if not files:
        myText += "No new files processed."
    return str(myText)

display(Markdown(createMessage()))


In [ ]:
dirs_remote = ["https://alice-tpc-qc.web.cern.ch/reports/"+di[di.find("202"):] for di in dirs]
dirs_remote

In [ ]:
import requests        
#actual code for sending
#headers = {'Content-Type': 'application/json'}
#values = { "text": "**Please ignore** This is a test message. [Link to TPC Async](https://alice-tpc-qc.web.cern.ch/)" }
headers = {'Content-Type': 'application/json',}
myMessage = "This is a **test** message please ignore. [Link to TPC QC](https://alice-tpc-qc.web.cern.ch/)"
values = '{ "text": \"'+myMessage+'\" }'

# Uncomment lines below to actually send message
#response = requests.post("https://mattermost.web.cern.ch/hooks/krtdox9rbtgsxgqif3ijy51y8c",headers=headers, data=values)
#print(response)